# M1 useful nodes
This notebook identifies M1 nodes (attention heads and MLPs) that, when ablated, cause a decrease in model prediction accuracy. These nodes are needed (aka useful) for accurate predictions.


This notebook was:
- Developed on Google Colab using an **T4**
- Runs with M1 (TinyStories) with base/CS1/CS2/CS3 models.
- Requires a GITHUB_TOKEN secret to access Martian TinySQL code repository.
- Requires a HF_TOKEN secret to access Martian HuggingFace repository.

This notebook relies on the nnsight library. Useful background:
- https://nnsight.net/notebooks/tutorials/walkthrough/#Batching
- https://nnsight.net/notebooks/tutorials/walkthrough/#Looping

This notebook relies on the https://github.com/PhilipQuirke/quanta_mech_interp library for graphing useful nodes.

# Import libraries
Imports standard libraries. Do not read.

In [14]:
# https://nnsight.net/
# Access 0.4 prerelease version (as at Dec 2024)
#!pip install nnsight==0.4.0.dev0
!pip install -U nnsight

In [15]:
from IPython.display import clear_output
import einops
import torch
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"
import matplotlib.pyplot as plt
import tqdm.auto as tqdm

import nnsight
from nnsight import LanguageModel

In [16]:
from getpass import getpass
from google.colab import userdata
import gc
import weakref

In [17]:
!pip install --upgrade git+https://github.com/PhilipQuirke/quanta_mech_interp.git
import QuantaMechInterp as qmi

  Cloning https://github.com/PhilipQuirke/quanta_mech_interp.git to /tmp/pip-req-build-yaa0wjir
  Running command git clone --filter=blob:none --quiet https://github.com/PhilipQuirke/quanta_mech_interp.git /tmp/pip-req-build-yaa0wjir
  Resolved https://github.com/PhilipQuirke/quanta_mech_interp.git to commit 7a53d2b4cf91a41532b2f8437fe43e58febca204
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
github_token = userdata.get("GITHUB_TOKEN")

# Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/TinySQL.git

import TinySQL as qts

  Cloning https://****@github.com/withmartian/TinySQL.git to /tmp/pip-req-build-9wgo544c
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/withmartian/TinySQL.git' /tmp/pip-req-build-9wgo544c
  Resolved https://****@github.com/withmartian/TinySQL.git to commit 8c4ec093c1f14305ee081a3f1b54b7cf44c7cda2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Select model and command set to investigate


In [19]:
model_num = 1                 # 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 1                    # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3

# Load model

In [ ]:
model = qts.load_tinysql_model(model_num, cs_num, auth_token=userdata.get("HF_TOKEN"))
clear_output()
print(model)

In [ ]:
N_LAYERS, N_HEADS, D_MODEL, D_HEAD = qts.get_model_sizes(model_num, model)

In [ ]:
model_hf_name = qts.sql_interp_model_location(model_num, cs_num)

# Singleton QuantaTool "main" configuration class. qmi.AlgoConfig is derived from the chain qmi.UsefulConfig > qmi.ModelConfig
cfg = qmi.AlgoConfig()
cfg.repo_name, cfg.model_name = model_hf_name.rsplit("/", 1)
cfg.main_model = model
cfg.n_layers = N_LAYERS
cfg.n_heads = N_HEADS
cfg.d_model = D_MODEL
cfg.d_head = D_HEAD
cfg.set_seed(673023)

print("cfg.repo_name="+cfg.repo_name, "cfg.model_name="+cfg.model_name)

# Generate test data and experiment runs

In [ ]:
# Generate a batch of prompts with 3 field names
def generate_batch(batch_size):
    cfg.batch_size = batch_size

    return qts.generate_csn(batch_size=batch_size, csn=max(1,cs_num), min_cols=3, max_cols=3)

In [ ]:
# Check if the model can correctly predict the question's answer
def run_attention_experiment(run_layer_idx, run_head_idx, run_token_idx, all_tokens, run_prompt_tokens, run_generate, make_changes = True, extra_tokens=0):

    assert len(all_tokens) == run_prompt_tokens + run_generate

    if run_generate == 0:
        return True, "", ""

    start = run_head_idx * D_HEAD
    end = (run_head_idx + 1) * D_HEAD

    with model.generate(all_tokens[:run_prompt_tokens], max_new_tokens=run_generate + extra_tokens, pad_token_id=model.tokenizer.eos_token_id):

        if make_changes:
            model.transformer.h[run_layer_idx].output[0][:, run_token_idx, start:end] = 0

        final_output = model.generator.output.save()

    # Did the output change?
    decoded_input = model.tokenizer.decode(all_tokens, skip_special_tokens=True)
    decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)
    same = decoded_input==decoded_output

    return same, decoded_input, decoded_output

In [ ]:
# Check if the model can correctly predict the question's answer and print result
def check_experiment(prompt, answer, all_tokens, run_prompt_tokens, run_generate, make_changes = True, show_same = True):

    same, decoded_input, decoded_output = run_attention_experiment(0, 0, 0, all_tokens, run_prompt_tokens, run_generate, make_changes, extra_tokens=100)

    if not same:
        print( "Ignoring example that model doesn't predict correctly:")

    if (not same) or show_same:
        print("Prompt ("+str(run_prompt_tokens)+") :", prompt.replace('\n', '\\n'))
        print("Answer ("+str(run_generate)+") :", answer.replace('\n', '\\n'))
        print("Output:", decoded_output.replace('\n', '\\n'))
        print()

    return same, decoded_input, decoded_output

In [ ]:
# Check that generators work
def check_generator_on_clean_input(examples):
    num_good = 0
    for example in examples:
        prompt = example.get_alpaca_prompt()
        answer = example.sql_statement

        prompt_tokens = model.tokenizer(prompt)["input_ids"]
        answer_tokens = model.tokenizer(answer)["input_ids"]
        all_tokens = prompt_tokens + answer_tokens

        num_answer_tokens = len(answer_tokens)
        num_prompt_tokens = len(prompt_tokens)

        same, _, decoded_output = check_experiment(prompt, answer, all_tokens, num_prompt_tokens, num_answer_tokens, make_changes = False, show_same = False)
        if same:
            num_good += 1

    # Almost all examples should succeed
    assert num_good >= len(examples) - 1


generator = qts.CorruptFeatureTestGenerator(model_num=model_num, cs_num=cs_num, tokenizer=model.tokenizer, use_novel_names=True)
check_generator_on_clean_input( generator.generate_feature_examples(qts.ENGFIELDNAME, 6) )

check_generator_on_clean_input( qts.generate_csn(batch_size=6, csn=max(1,cs_num), min_cols=3, max_cols=3) )

In [ ]:
# Return a list of experiments to run
def get_experiment_list(examples, by_attention_head):
    run_list = []
    max_prompt_tokens = 0
    max_answer_tokens = 0
    max_tokens = 0
    show_examples = 3

    for example in examples:
        prompt = example.get_alpaca_prompt()
        answer = example.sql_statement

        prompt_tokens = model.tokenizer(prompt)["input_ids"]
        answer_tokens = model.tokenizer(answer)["input_ids"]
        all_tokens = prompt_tokens + answer_tokens

        num_answer_tokens = len(answer_tokens)
        num_prompt_tokens = len(prompt_tokens)
        num_tokens = len(all_tokens)
        assert num_tokens == num_prompt_tokens + num_answer_tokens

        max_prompt_tokens = max(max_prompt_tokens, num_prompt_tokens)
        max_answer_tokens = max(max_answer_tokens, num_answer_tokens)
        max_tokens = max(max_tokens, num_tokens)

        # Check that the model can correctly predict the question's answer
        same, _, decoded_output = check_experiment(prompt, answer, all_tokens, num_prompt_tokens, num_answer_tokens, make_changes = False, show_same = show_examples > 0)

        if not same:
            continue
        show_examples -= 1

        if by_attention_head:
            for layer_idx in range(N_LAYERS):
                for head_idx in range(N_HEADS):
                    for token_idx in range(num_tokens):
                        # Important logic:
                        # num_prompt_tokens and num_answer_tokens are the normal interpretation of the prompt/answer sizes.
                        # If ablating a token in the MIDDLE of the PROMPT, we provide ALL the prompt tokens, and generate num_answer_tokens.
                        # If ablating a token in the MIDDLE of the ANSWER, we increase the size of the "prompt" and decrease the generated tokens.
                        exp_prompt_tokens = max(num_prompt_tokens, token_idx+1)
                        exp_num_generate = num_tokens - exp_prompt_tokens

                        assert num_tokens == exp_prompt_tokens + exp_num_generate

                        run_list.append([layer_idx, head_idx, token_idx, all_tokens, exp_prompt_tokens, exp_num_generate])
        else:
            for token_idx in range(num_prompt_tokens):
                run_list.append([prompt, answer, token_idx, prompt_tokens])

    return run_list, max_prompt_tokens, max_answer_tokens, max_tokens

# Which token positions are useful?
This information is used to shrink the size of search spaces in following sections. For the SQL model all token positions are useful

In [ ]:
N_BATCH = 50

def run_token_experiments( ):
    examples = generate_batch(N_BATCH)
    run_list, max_prompt_tokens, max_good_answer_tokens, max_tokens = get_experiment_list(examples, False)
    cfg.initialize_token_positions( max_prompt_tokens, max_good_answer_tokens, True )
    num_exps = len(run_list)

    print("N_BATCH="+str(N_BATCH), "max_prompt_tokens="+str(max_prompt_tokens), "max_good_answer_tokens="+str(max_good_answer_tokens), "max_tokens="+str(max_tokens), "num_exps="+str(num_exps))

    try_results = np.zeros(max_tokens, dtype=int)
    fail_results = np.zeros(max_tokens, dtype=int)

    for item_num in tqdm.tqdm(range(num_exps)):

        run_item = run_list[item_num]
        run_prompt, run_answer, run_token_idx, prompt_tokens = run_item

        with model.generate(prompt_tokens, max_new_tokens=max_good_answer_tokens, pad_token_id=model.tokenizer.eos_token_id) :

            # Zero out just the portion of the output corresponding to this token position
            for run_layer_idx in range(N_LAYERS):
                model.transformer.h[run_layer_idx].output[0][:, run_token_idx, :] = 0

            final_output = model.generator.output.save()

        decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)

        # Did the output change?
        if run_prompt + run_answer != decoded_output:
            fail_results[run_token_idx] += 1
        try_results[run_token_idx] += 1

    # Compute the failure rate as percentage
    failure_rate = (1.0 * fail_results / (try_results + 1e-10)) * 100
    failure_rate = np.round(failure_rate, 2)

    return max_tokens, fail_results, failure_rate

g_max_tokens, g_token_fail_results, g_token_failure_rate = run_token_experiments()

In [ ]:
print( "Useful token positions:" )
for token_idx in range(len(g_token_failure_rate)):
    if g_token_failure_rate[token_idx] > 0 :
        cfg.add_useful_position(token_idx)
        print( "Position:", token_idx, "    % Fails:", g_token_failure_rate[token_idx], "    # Fails:", g_token_fail_results[token_idx] )

In [ ]:
#cfg.calc_position_failures_map(g_token_fail_results.tolist())
#qmi.save_plt_to_file(cfg=cfg, full_title="Failures When Position Ablated")
#plt.show()

# Which token+layer+attention head nodes are useful?

In [ ]:
N_BATCH = 10

def run_attention_experiments():
    show_diff = True
    examples = generate_batch(N_BATCH)
    run_list, max_prompt_tokens, max_good_answer_tokens, max_tokens = get_experiment_list(examples, True)
    num_exps = len(run_list)

    print("N_BATCH="+str(N_BATCH), "max_prompt_tokens="+str(max_prompt_tokens), "max_answer_tokens="+str(max_good_answer_tokens), "max_tokens="+str(max_tokens), "num_exps="+str(num_exps))

    try_results = np.zeros((N_LAYERS, N_HEADS, max_tokens), dtype=int)
    fail_results = np.zeros((N_LAYERS, N_HEADS, max_tokens), dtype=int)

    for item_num in tqdm.tqdm(range(num_exps)):

        run_item = run_list[item_num]
        run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate = run_item

        same, decoded_input, decoded_output = run_attention_experiment(run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate)

        if not same:
            fail_results[run_layer_idx, run_head_idx, run_token_idx] += 1
            if show_diff:
                print("Failure when intervening:", "Layer="+str(run_layer_idx), "Head="+str(run_head_idx), "Pos="+str(run_token_idx), "NumPrompts="+str(num_prompt_tokens), "NumGenerate="+str(num_generate))
                print("Input :", decoded_input.replace('\n', '\\n'))
                print("Output:", decoded_output.replace('\n', '\\n'))
                show_diff = False
        try_results[run_layer_idx, run_head_idx, run_token_idx] += 1


    # Compute the failure rate as percentage
    failure_rate = (1.0 * fail_results / (try_results + 1e-10)) * 100
    failure_rate = np.round(failure_rate, 2)

    return max_tokens, fail_results, failure_rate, try_results

g_max_tokens, g_attn_failure_results, g_attn_failure_rate, g_attn_try_results = run_attention_experiments()

In [ ]:
cfg.useful_nodes = qmi.UsefulNodeList()
for layer_idx in range(N_LAYERS):
    for head_idx in range(N_HEADS):
        for token_idx in range(g_max_tokens):
            fail_perc = int(g_attn_failure_rate[layer_idx, head_idx, token_idx])
            if fail_perc > 0 :
                # Add percentage failure quanta
                node_location = qmi.NodeLocation(token_idx, layer_idx, True, head_idx)
                cfg.add_useful_node_tag( node_location, qmi.QType.FAIL.value, str(fail_perc) )

cfg.useful_nodes.sort_nodes()

In [ ]:
for layer_idx in range(N_LAYERS):
    title = "TinySQL Percentage of Output Changes by Zeroing Activations in Layer " + str(layer_idx)
    plt.imshow(g_attn_failure_rate[layer_idx], cmap="viridis", aspect="auto")
    plt.colorbar(label="Percentage Change")
    plt.xlabel("Token Position")
    plt.ylabel("Attention Head")
    plt.title(title)
    plt.show()
    qmi.save_plt_to_file(cfg=cfg, full_title=title)

In [ ]:
# cfg.useful_nodes.print_node_tags()

In [ ]:
title = "TinySQL useful attention heads"
ax1, quanta_results, num_results = qmi.calc_quanta_map(
    cfg, True, 6,
    cfg.useful_nodes, qmi.QType.FAIL.value, "", qmi.get_quanta_fail_perc,
    combine_identical_cells=False)

if num_results > 0:
    if cfg.graph_file_suffix > "":
        print("Saving quanta map:", title)
        qmi.save_plt_to_file(cfg=cfg, full_title=title)
    else:
        ax1.set_title(title + ' ({} nodes)'.format(len(quanta_results)))

    plt.show()

In [ ]:
# Serialize and save the useful nodes list to a temporary CoLab file in JSON format. Manually download.
useful_node_json_filename = 'tinysql_bm' + str(model_num) + "_cs" + str(cs_num) + '_useful_nodes.json'
print( "Saving useful node list with behavior tags:", useful_node_json_filename)
cfg.useful_nodes.save_nodes(useful_node_json_filename)

#TODO: Auto save to Martian wandb